In [1]:
%pip install torch zkstats==0.1.8


[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    setup,
    verifier_verify,
)
from zkstats.computation import computation_to_model

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"
precal_witness_path = f"{output_dir}/precal_witness.json"

In [3]:
# Data provided shared by the data provider beforehand

def get_data_shape(data_path: str) -> dict[str, int]:
    """
    Get the shape of the data from the data file.

    Parameters:
    - data_path (str): The path to the data file.

    Returns:
    - shape_info (dict): A dictionary where keys are column names and values are the number of elements (shape).
    """
    with open(data_path, 'r') as f:
        data = json.load(f)
    shape_info = {col: len(data[col]) for col in data.keys()}
    return shape_info

data_shape = get_data_shape(data_path)
print(data_shape)
data_commitment_path = f"{output_dir}/data_commitment.json"

{'x': 7, 'y': 7}


## Columns selected by the user

In [4]:
# FIXME: this should be provided by users
# selected_columns = ["x", "y"]
selected_columns = ["x"]

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, args: Args):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.cat([out_0.unsqueeze(0), out_1.unsqueeze(0)]).reshape(-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [5]:
# This is just a dummy computation. Replace it with user's computation
import torch
from zkstats.computation import State, Args

def computation(state: State, args: Args):
    x = args["x"]
    return state.mean(x), state.median(x)

Generate settings and setup with user's computation.

In [6]:
print(f"{selected_columns=}")
_, model = computation_to_model(computation, precal_witness_path, isProver=True, selected_columns=selected_columns)
prover_gen_settings(
    data_path,
    selected_columns,
    comb_data_path,
    model,
    model_onnx_path,
    [3],
    "resources",
    settings_path,
)

# Determine which srs to use with the logrows in the settings.json
with open(settings_path, "r") as f:
    settings = json.load(f)
logrows = int(settings["run_args"]["logrows"])
srs_path = f'~/.ezkl/srs/kzg{logrows}.srs'

selected_columns=['x']


/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where_1): Y has inconsistent type tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 3, param_scale: 3, scale_input_multiplier: 10) ------------->

+---------------+--------------+------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error | max_error  | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+--------------+------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.0023816426 | 0.04285431   | 0.04285431 | -0.049999237 | 0.030951181    | 0.04285431       | 0.049999237   | 0             | 0.0014454719       | -0.0000836013      | 0.0006378884           |
+---------------+--------------+------------+--------------+----------------+------------------

scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-856,754],"logrows":11,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":1312,"total_assignments":502,"total_const_size":212,"model_instance_shapes":[[1],[1],[1]],"model_output_scales":[0,3,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[1312,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs","ReLU",{"Floor":{"scale":16.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1720109952300}


In [7]:
setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

==== setting up ezkl ====
Time setup: 0.7527332305908203 seconds


Generate proof with your data and user's computation.

In [8]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)

==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 51.5
==== Generating Proof ====
proof:  {'instances': [['3bea079e2a500a3285d05fd174263a5ffd78c2dd63131c52abd2a199f786f601', '0100000000000000000000000000000000000000000000000000000000000000', '9c01000000000000000000000000000000000000000000000000000000000000', '7201000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1151a27e4fff31b51cbc475a9a4b9cce6cf662e530e1f9ba74495ef85b6f244a2672447acedbc2a14098e04c347828f398f83d7488da7249043625b25769c70a120102370a549dcffb5c6da3efb1a7f0890ac67dc8fddc8c21085314f7e5cc1b2cb82377d5297b8fe2973b9970340588d5597a083223493029802bf562c19a8b0c87b0c4b645666f4cc0ef617e6da951f76ac252ed72c573002f3330104b553128b546384da61dddb10d3f8c92fe005bb6ecd4847bf5cccedab4f0bb9a96598c04e97e41c660d5d060ce9d2fa700fded05010375c9bbf7d5b58374d098e4a2dc090b2fa4b07184594f89bbaa174495e1d5f2ffc1c49d9f080a6b3eac3214fbb01416c736561f88f82c4b6618967d675eb0d95f4683efa785310817a81c3407360b845cfe18aa

## Verify the proof to ensure it is correct
NOTE: The following section is to illustrate what should be done on the user (data consumer) side. This step is not required by the data provider.

In [9]:
# verify the proof
verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)

[51.5, 46.25]

Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [10]:
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)
print("Srs path:\t\t", srs_path)

Model onnx:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.onnx
Settings:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/settings.json
Proof:			 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.pf
Verification key:	 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.vk
Srs path:		 ~/.ezkl/srs/kzg11.srs
